In [1]:
import sys
import os
import torch
from torch import optim
from time import time

In [2]:
tile2vec_dir = '/home/agupta21/gcloud/231n_gitproject'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [3]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet

In [4]:
from src.training import prep_triplets, train_triplet_epoch

# Step 1. Download triplets from bucket

Using the download link, unzip triplets into the directory /tile2vec/data/triplets.

# Step 2. Set up dataloader

In [5]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()
print(cuda)

True


Set up the dataloader for training.

In [11]:
# Change these arguments to match your directory and desired parameters
img_type = 'naip'
tile_dir = '/home/agupta21/gcloud/231n_gitproject/data/triplets/'
bands = 4
augment = False
batch_size = 50
shuffle = True
num_workers = 4
n_triplets = 108700

In [12]:
dataloader = triplet_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


# Step 3. Set up TileNet

In [13]:
in_channels = bands
z_dim = 512

In [14]:
TileNet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
TileNet.train()
if cuda: TileNet.cuda()
print('TileNet set up complete.')

TileNet set up complete.


Set up optimizer.

In [15]:
lr = 1e-3
optimizer = optim.Adam(TileNet.parameters(), lr=lr, betas=(0.5, 0.999))

# Step 4. Train model!

In [16]:
epochs = 2
margin = 10
l2 = 0.01
print_every = 10000
save_models = True

Define the directory for saving models.

In [17]:
model_dir = '/home/agupta21/gcloud/231n_gitproject/models/'
if not os.path.exists(model_dir): os.makedirs(model_dir)

In [19]:
t0 = time()
#with open(results_fn, 'w') as file:


plot_list = []
print('Begin training.................')
for epoch in range(0, epochs):
    (avg_loss, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
        TileNet, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
        print_every=print_every, t0=t0)
    plot_list.append((epoch,avg_loss))
    

Begin training.................
Epoch 1: [10000/108700 (9%)], Avg loss: 6.5402
Epoch 1: [20000/108700 (18%)], Avg loss: 5.6775
Epoch 1: [30000/108700 (28%)], Avg loss: 5.5063
Epoch 1: [40000/108700 (37%)], Avg loss: 5.3885
Epoch 1: [50000/108700 (46%)], Avg loss: 5.0784
Epoch 1: [60000/108700 (55%)], Avg loss: 5.0233
Epoch 1: [70000/108700 (64%)], Avg loss: 4.9560
Epoch 1: [80000/108700 (74%)], Avg loss: 4.8924
Epoch 1: [90000/108700 (83%)], Avg loss: 4.9374
Epoch 1: [100000/108700 (92%)], Avg loss: 4.7877
Finished epoch 1: 2561.881s
  Average loss: 5.2361
  Average l_n: 4.2330
  Average l_d: -12.9484
  Average l_nd: -8.7153



In [20]:
# Save model after last epoch
if save_models:
    model_fn = os.path.join(model_dir, 'testing_run2.ckpt')
    torch.save(TileNet.state_dict(), model_fn)